In [2]:
import os


os.chdir("..")
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [21]:
from lib import *

In [50]:
import numpy as np
import pandas as pd
from PIL import Image
import torch
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
from tqdm import tqdm
import safitty

In [4]:
from src import prepare_for_inference, imread, GlobalDice, GlobalIoU
from src.dataset import SegmentationDataset

In [5]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

setup = prepare_for_inference(
    "./params/pan_eff_net_b0.yml",
    "./logs/lightning_logs/version_1/checkpoints/epoch=39.ckpt")

model = setup["model"]
model = model.to(device)

preproc = setup["preprocessing_function"]
params = safitty.load("params/pan_eff_net_b0.yml")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/checkpoints/efficientnet-b0-355c32eb.pth


rle_masks

In [51]:
path = "data/cig_butts/val/images/"
pred = pd.read_csv("data/pred_val_template.csv")

for image_name in tqdm(os.listdir(path)):
    image = np.array(Image.open(os.path.join(path, image_name)))[:,:,:3]
    image = preproc(image).unsqueeze(0)
    image = image.to(device)
    with torch.no_grad():
        pred_mask = torch.sigmoid(model(image)).cpu().numpy()
    image_id = int(image_name.split(".")[0])
    mask = (pred_mask[0][0] > 0.5).astype(np.int8) * 255
    rle_mask = encode_rle(mask)
    pred.loc[image_id, "rle_mask"] = rle_mask

100%|██████████| 200/200 [00:07<00:00, 26.64it/s]


In [53]:
pred.to_csv("data/pred_val_template.csv", index=False)

HTML render

In [54]:
path = "data/cig_butts/real_test/"
image_names = os.listdir(path)
pred_masks = []

for image_name in tqdm(image_names):
    image = np.array(Image.open(os.path.join(path, image_name)))[:,:,:3]
    image = preproc(image).unsqueeze(0)
    image = image.to(device)
    with torch.no_grad():
        pred_mask = torch.sigmoid(model(image)).cpu().numpy()
    mask = (pred_mask[0][0] > 0.5).astype(np.int8) * 255
    pred_masks.append(mask)
    
_ = get_html(
    [os.path.join(path, image_name) for image_name in image_names],
    pred_masks,
    path_to_save="results/example"
)

100%|██████████| 10/10 [00:00<00:00, 30.35it/s]


---